In [9]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    features_df = pd.read_csv(resource_dir / f"templates_{config_dict['retrofit_scenario']}.csv")

    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for idf in idfs:
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors, idf_cluster

In [12]:
config_dict = {
    "building_id": "1",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
    "retrofit_scenario": "highdhw",
    "schedules_scenaro": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1980,
    "lab_weight": 0.8
}


sb_weight_map = {
    "Perimeter": "PerimeterAreaWeight",
    "Core": "CoreAreaWeight"
}


res, err, idf_cluster = run_single_building(config_dict, 
                               weight_map = sb_weight_map,
                               output_dir = Path(os.getcwd()) / "eplus", 
                               idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs"),
                               update_epsjon=True
                               )

C:\Users\zoelh\AppData\Local\Temp\ipykernel_466540\2271334121.py:42: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting DHWSupplyTemp to 65.0
INFO:EPJSON:Setting WindowUValue to 5.78
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting VentilationPerPerson to 0.000794118
INFO:EPJSON:Setting LightsSchedule to 30HrWk
INFO:EPJSON:Setting LightingPowerDensity to 10.0
INFO:EPJSON:Setting VentilationMode to OccupancySchedule
INFO:EPJSON:Setting EquipmentPowerDensity to 10.0
INFO:EPJSON:Setting FacadeRValue to 1.17266436
INFO:EPJSON:Setting RoofRValue to 2.591349481
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting EconomizerSettings to NoEconomizer
INFO:EPJSON:Setting VentilationPerArea to 0.000794118
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting HeatingAvailability to A

BUILDING CLUSTER


Fetching epjsons...: 100%|██████████| 4/4 [00:00<00:00, 48.76it/s]


SIMULATING CLUSTER


100%|██████████| 4/4 [00:06<00:00,  1.72s/it]
INFO:BATCH:Simulated all idfs.
INFO:BATCH:Fetching results for 1


PROCESSING CLUSTER RESULTS


INFO:BATCH:There are 4 shoeboxes for 1
Fetching building results: 100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


In [11]:
res['1'].sum()

C:\Users\zoelh\AppData\Local\Temp\ipykernel_466540\4247261114.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  res['1'].sum()


Zone Lights Electricity Energy                                                               132441.832735
Electric Equipment Electricity Energy                                                        116010.601343
Zone Ideal Loads Supply Air Total Heating Energy                                             470865.027626
Zone Ideal Loads Supply Air Total Cooling Energy                                             203060.558776
Water Use Equipment Heating Energy                                                          2378993.044731
Zone Lights Electricity Energy_norm                                                              26.570394
Electric Equipment Electricity Energy_norm                                                       23.273971
Zone Ideal Loads Supply Air Total Heating Energy_norm                                            94.464634
Zone Ideal Loads Supply Air Total Cooling Energy_norm                                            40.737877
Water Use Equipment Heating Energy_no

In [13]:
res['1'].sum()

C:\Users\zoelh\AppData\Local\Temp\ipykernel_466540\4247261114.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  res['1'].sum()


Zone Lights Electricity Energy                                                               132441.832735
Electric Equipment Electricity Energy                                                        116010.601343
Zone Ideal Loads Supply Air Total Heating Energy                                             463734.755814
Zone Ideal Loads Supply Air Total Cooling Energy                                             195331.218061
Water Use Equipment Heating Energy                                                         23789930.447306
Zone Lights Electricity Energy_norm                                                              26.570394
Electric Equipment Electricity Energy_norm                                                       23.273971
Zone Ideal Loads Supply Air Total Heating Energy_norm                                            93.034164
Zone Ideal Loads Supply Air Total Cooling Energy_norm                                            39.187221
Water Use Equipment Heating Energy_no

In [7]:
# res['1']

,Zone Lights Electricity Energy,Zone Electric Equipment Electricity Energy,Zone Ideal Loads Supply Air Total Heating Energy,Zone Ideal Loads Supply Air Total Cooling Energy,Water Use Equipment Heating Energy,Zone Lights Electricity Energy_norm,Zone Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,ModelArea,DateTime,BuildingId
0,13.374925,26.749849,677.335619,0.041169,0.000000,0.000997,0.001994,0.050501,0.000003,0.000000,13412.436331,2018-01-01 00:00:00,1
1,13.374925,26.749849,602.073929,0.029170,0.000000,0.000997,0.001994,0.044889,0.000002,0.000000,13412.436331,2018-01-01 01:00:00,1
2,13.374925,26.749849,616.400337,0.026316,0.000000,0.000997,0.001994,0.045957,0.000002,0.000000,13412.436331,2018-01-01 02:00:00,1
3,13.374925,26.749849,626.058339,0.023457,0.000000,0.000997,0.001994,0.046677,0.000002,0.000000,13412.436331,2018-01-01 03:00:00,1
4,13.374925,26.749849,629.376412,0.013727,0.000000,0.000997,0.001994,0.046925,0.000001,0.000000,13412.436331,2018-01-01 04:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,13.374925,26.749849,470.946408,0.120331,887.642601,0.000997,0.001994,0.035113,0.000009,0.066181,13412.436331,2018-12-31 19:00:00,1
8756,13.374925,26.749849,472.692196,0.108328,0.000000,0.000997,0.001994,0.035243,0.000008,0.000000,13412.436331,2018-12-31 20:00:00,1
8757,13.374925,26.749849,485.266959,0.101161,0.000000,0.000997,0.001994,0.036180,0.000008,0.000000,13412.436331,2018-12-31 21:00:00,1
8758,13.374925,26.749849,482.152300,0.092469,0.000000,0.000997,0.001994,0.035948,0.000007,0.000000,13412.436331,2018-12-31 22:00:00,1
